In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import OneHotEncoder
from prophet import Prophet

# # Load the dataset
# file_path = "your_file.csv"  # Replace with actual file path
df = pd.read_csv('ecom_sales_dnd.csv')

c:\Users\SudeshKumar\anaconda3\envs\forecast-new2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
df.head()

,Product Name,Customer ID,Order City,Quantity,Total Sales Value,Order Status,Timestamp,Unit Price
0,Keyboard,33f8d812-290e-4b71-9db7-8f27aaa81b0f,Seattle,6,911.10,Pending,2/23/2024,151.85
1,Tablet,5a7c8b80-e2f7-4c1a-bda2-cc0109a69e7f,Miami,6,495.06,Pending,5/14/2024,82.51
2,Camera,1e13337b-aeeb-4a5a-8ced-8b6e7683f7ae,Houston,2,11.72,Cancelled,6/28/2024,5.86
3,Smartphone,d2e022b4-ef09-4e13-921b-bba16b30c892,Dallas,6,1610.52,Cancelled,2/10/2024,268.42
4,Monitor,07aec762-dc9f-4688-a7e5-c53d443b8073,Houston,3,611.70,Pending,5/4/2024,203.90


In [3]:


# Convert timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Aggregate sales by month and product
df['Month'] = df['Timestamp'].dt.to_period('M').dt.to_timestamp()

In [5]:
df['Month2']=df['Timestamp'].dt.month_name()

In [6]:
df.head()

,Product Name,Customer ID,Order City,Quantity,Total Sales Value,Order Status,Timestamp,Unit Price,Month,Month2
0,Keyboard,33f8d812-290e-4b71-9db7-8f27aaa81b0f,Seattle,6,911.10,Pending,2024-02-23,151.85,2024-02-01,February
1,Tablet,5a7c8b80-e2f7-4c1a-bda2-cc0109a69e7f,Miami,6,495.06,Pending,2024-05-14,82.51,2024-05-01,May
2,Camera,1e13337b-aeeb-4a5a-8ced-8b6e7683f7ae,Houston,2,11.72,Cancelled,2024-06-28,5.86,2024-06-01,June
3,Smartphone,d2e022b4-ef09-4e13-921b-bba16b30c892,Dallas,6,1610.52,Cancelled,2024-02-10,268.42,2024-02-01,February
4,Monitor,07aec762-dc9f-4688-a7e5-c53d443b8073,Houston,3,611.70,Pending,2024-05-04,203.90,2024-05-01,May


NameError: name 'df' is not defined

In [7]:
monthly_sales = df.groupby(['Month', 'Product Name'])['Total Sales Value'].sum().reset_index()

In [10]:
monthly_sales

,Month,Product Name,Total Sales Value
0,2024-02-01,Camera,64611.20
1,2024-02-01,Gaming Console,64261.12
2,2024-02-01,Headphones,67810.70
3,2024-02-01,Keyboard,68566.93
4,2024-02-01,Laptop,59894.51
...,...,...,...
123,2025-02-01,Monitor,432.57
124,2025-02-01,Mouse,2356.32
125,2025-02-01,Smartphone,4775.51
126,2025-02-01,Smartwatch,2696.75


In [12]:
# Rename columns for Prophet
monthly_sales.rename(columns={'Month': 'ds', 'Total Sales Value': 'y', 'Product Name': 'product'}, inplace=True)

In [13]:
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_products = encoder.fit_transform(monthly_sales[['product']])

In [15]:
len(encoded_products)

128

In [16]:
encoded_df = pd.DataFrame(encoded_products, columns=encoder.get_feature_names_out(['product']))

In [18]:
encoded_df.head()

,product_Gaming Console,product_Headphones,product_Keyboard,product_Laptop,product_Monitor,product_Mouse,product_Smartphone,product_Smartwatch,product_Tablet
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [19]:
monthly_sales = pd.concat([monthly_sales, encoded_df], axis=1)

In [20]:
monthly_sales

,ds,product,y,product_Gaming Console,product_Headphones,product_Keyboard,product_Laptop,product_Monitor,product_Mouse,product_Smartphone,product_Smartwatch,product_Tablet
0,2024-02-01,Camera,64611.20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-02-01,Gaming Console,64261.12,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-02-01,Headphones,67810.70,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-02-01,Keyboard,68566.93,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-02-01,Laptop,59894.51,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
123,2025-02-01,Monitor,432.57,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
124,2025-02-01,Mouse,2356.32,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
125,2025-02-01,Smartphone,4775.51,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
126,2025-02-01,Smartwatch,2696.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
import pandas as pd
import pickle
from sklearn.preprocessing import OneHotEncoder
from prophet import Prophet

# # Load the dataset
# file_path = "your_file.csv"  # Replace with actual file path
df = pd.read_csv('ecom_sales_dnd.csv')

# Convert timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Aggregate sales by month and product
df['Month'] = df['Timestamp'].dt.to_period('M').dt.to_timestamp()
monthly_sales = df.groupby(['Month', 'Product Name'])['Total Sales Value'].sum().reset_index()

# Rename columns for Prophet
monthly_sales.rename(columns={'Month': 'ds', 'Total Sales Value': 'y', 'Product Name': 'product'}, inplace=True)

# Train the one-hot encoder
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_products = encoder.fit_transform(monthly_sales[['product']])
encoded_df = pd.DataFrame(encoded_products, columns=encoder.get_feature_names_out(['product']))

# Merge encoded features
monthly_sales = pd.concat([monthly_sales, encoded_df], axis=1)

# Save the encoder for future use
with open("encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

# Train the Prophet model
model = Prophet()

# Add categorical regressors
for col in encoded_df.columns:
    model.add_regressor(col)

# Fit the model
model.fit(monthly_sales)

# Save the trained Prophet model
with open("monthly_sales_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model and Encoder saved successfully!")

In [ ]:
import pandas as pd
import pickle
 
# Load the trained Prophet model
with open("monthly_sales_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)
 
# Load the one-hot encoder
with open("encoder.pkl", "rb") as f:
    encoder = pickle.load(f)
 
# Function to make predictions
def predict_sales(product_name, months_ahead):
    # Get the last date from the trained dataset
    last_date = loaded_model.history['ds'].max()
 
    # Generate future dates (Monthly)
    future_dates = pd.date_range(start=last_date, periods=months_ahead, freq='ME')
    future_df = pd.DataFrame({'ds': future_dates})
 
    # One-hot encode product for prediction
    product_columns = encoder.get_feature_names_out(['product'])
    product_encoded = pd.DataFrame(0, index=[0], columns=product_columns)
 
    # Set the specific product column to 1
    product_col = f'product_{product_name}'
    if product_col in product_encoded.columns:
        product_encoded.loc[0, product_col] = 1
    else:
        raise ValueError(f"Product '{product_name}' not found in trained model!")
 
    # Repeat product encoding for all future months
    product_encoded = pd.concat([product_encoded.iloc[0:1]] * len(future_df), ignore_index=True)
 
    future_encoded = pd.concat([future_df, product_encoded], axis=1)
 
    # Make predictions
    forecast = loaded_model.predict(future_encoded)
 
    # Prepare response JSON
    forecast_json = [
        {"product_name": product_name, "time_period": row["ds"].strftime("%Y-%m-%d"), "forecast_sales": row["yhat"]}
        for _, row in forecast.iterrows()
    ]
    return forecast_json
 
 
user_input = ["Tablet", 3]  # Forecast for "Monitor" for 3 months
predicted_sales = predict_sales(user_input[0], user_input[1])
# print(predicted_sales)
import json
print(json.dumps(predicted_sales, indent=2))